In [41]:
import numpy as np
import json

from path_lib import inverse_kinematics
from path_tool import gen_posture
from path_tool import gen_walk_path, gen_fastwalk_path, gen_turn_path, gen_climb_path
from path_tool import (
    gen_rotatex_path,
    gen_rotatey_path,
    gen_rotatez_path,
    gen_twist_path,
)
from path_tool import gen_standup_path

servo_min = 102
servo_max = 512
servo_range = servo_max - servo_min

In [42]:
with open("config.json", "r", encoding="utf-8") as read_file:
    config = json.load(read_file)

In [43]:
standby = gen_posture(60, 75, config)
laydown = gen_posture(25, 25, config)


angles = inverse_kinematics(standby, config)

np.round(angles / 180 * servo_range + servo_min)

array([[307., 375., 341.],
       [307., 239., 273.],
       [307., 239., 273.],
       [307., 239., 273.],
       [307., 375., 341.],
       [307., 375., 341.]])

In [44]:
lut_standby = standby[np.newaxis, :, :]

lut_walk_0 = gen_walk_path(standby, direction=0)
lut_walk_180 = gen_walk_path(standby, direction=180)

lut_walk_r45 = gen_walk_path(standby, direction=315)
lut_walk_r90 = gen_walk_path(standby, direction=270)
lut_walk_r135 = gen_walk_path(standby, direction=225)

lut_walk_l45 = gen_walk_path(standby, direction=45)
lut_walk_l90 = gen_walk_path(standby, direction=90)
lut_walk_l135 = gen_walk_path(standby, direction=135)

lut_fast_forward = gen_fastwalk_path(standby, g_steps=28)
lut_fast_backward = gen_fastwalk_path(standby, g_steps=28, reverse=True)

lut_turn_left = gen_turn_path(standby, direction="left")
lut_turn_right = gen_turn_path(standby, direction="right")

lut_climb_forward = gen_climb_path(standby, reverse=False)
lut_climb_backward = gen_climb_path(standby, reverse=True)

lut_rotate_x = gen_rotatex_path(standby, g_steps=28, swing_angle=10, y_radius=10)
lut_rotate_y = gen_rotatey_path(standby, g_steps=28, swing_angle=10, x_radius=10)
lut_rotate_z = gen_rotatez_path(standby, g_steps=28, z_lift=7)

lut_twist = gen_twist_path(standby, g_steps=28)

lut_standup = gen_standup_path(standby, laydown, steps=28)

In [45]:
np.shape(lut_standup)

(28, 6, 3)

In [46]:
lut_standup[:,0,2]

array([  8.00102758,  -0.29517606,  -8.59137969, -16.88758333,
       -25.18378697, -33.4799906 , -41.77619424, -50.07239788,
       -58.36860151, -66.66480515, -66.66480515, -66.66480515,
       -66.66480515, -66.66480515, -66.66480515, -66.66480515,
       -66.66480515, -66.66480515, -66.66480515, -60.02348915,
       -54.18321388, -49.84840272, -47.54189788, -47.54189788,
       -49.84840272, -54.18321388, -60.02348915, -66.66480515])

In [47]:
var_name_list = [
    "lut_standby",
    "lut_walk_0",
    "lut_walk_180",
    "lut_walk_r45",
    "lut_walk_r90",
    "lut_walk_r135",
    "lut_walk_l45",
    "lut_walk_l90",
    "lut_walk_l135",
    "lut_fast_forward",
    "lut_fast_backward",
    "lut_turn_left",
    "lut_turn_right",
    "lut_climb_forward",
    "lut_climb_backward",
    "lut_rotate_x",
    "lut_rotate_y",
    "lut_rotate_z",
    "lut_twist",
    "lut_standup",
]

var_data_list = [
    lut_standby,
    lut_walk_0,
    lut_walk_180,
    lut_walk_r45,
    lut_walk_r90,
    lut_walk_r135,
    lut_walk_l45,
    lut_walk_l90,
    lut_walk_l135,
    lut_fast_forward,
    lut_fast_backward,
    lut_turn_left,
    lut_turn_right,
    lut_climb_forward,
    lut_climb_backward,
    lut_rotate_x,
    lut_rotate_y,
    lut_rotate_z,
    lut_twist,
    lut_standup,
]

In [48]:
fp = open("./motion.h", "w")

fp.write("/**\n")
fp.write(" * This is an automatically generated header, which includes motion path LUTs\n")
fp.write(" * \n")
fp.write(" * - Copyright (C) 2024 - PRESENT  rookidroid.com\n")
fp.write(" * - E-mail: info@rookidroid.com\n")
fp.write(" * - Website: https://rookidroid.com/\n")
fp.write(" */\n\n")

fp.write("#ifndef MOTION_H\n")
fp.write("#define MOTION_H\n\n")

for var_idx, var_name in enumerate(var_name_list):
    var = var_data_list[var_idx]

    fp.write("static int " + var_name + "_length = " + str(np.shape(var)[0]) + ";\n")

    for idx in range(0, np.shape(var)[0]):
        angles = inverse_kinematics(var[idx, :, :], config)
        path_walk_pwm = np.round(angles / 180 * servo_range + servo_min)
        path_walk_pwm = path_walk_pwm.astype(int)

        if idx == 0:
            fp.write(
                "static int " + var_name + "[" + str(np.shape(var)[0]) + "][6][3] = {{"
            )
        else:
            fp.write("                                  {")

        for r in range(0, 6):
            if r > 0:
                fp.write("                                   ")

            fp.write(
                "{"
                + str(path_walk_pwm[r, 0])
                + ", "
                + str(path_walk_pwm[r, 1])
                + ", "
                + str(path_walk_pwm[r, 2])
                + "}"
            )

            if r < 5:
                fp.write(",\n")
            else:
                fp.write("}")
        # fp.write("{"+str(path_walk_pwm[5, 0])+", " +str(path_walk_pwm[5, 1])+", "+str(path_walk_pwm[5, 2])+"}}")

        if idx == (np.shape(var)[0] - 1):
            fp.write("};\n\n")
        else:
            fp.write(",\n")

fp.write("#endif // MOTION_H\n")

fp.close()